In [2]:
import pandas as pd
import numpy as np
import scipy
import datetime

import plotly.graph_objects as go
import plotly.express as px
import chart_studio.plotly as py
from chart_studio.plotly import plot
import cufflinks as cf
cf.set_config_file(offline=True)
from plotly.subplots import make_subplots

import ipywidgets as widgets
from ipywidgets import interact, interact_manual, interactive, fixed
import matplotlib.pyplot as plt
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

plt.rcParams.update({'figure.max_open_warning': 0})
pd.set_option('mode.chained_assignment', None)
pd.options.display.max_rows = 200
pd.set_option("display.max_columns", 150)

build it

In [2]:
sheets = ['E26', 'E27', 'E28', 'K06', 'K10', 'K17', 'K18', 'K40']
master = pd.DataFrame()
for sheet in sheets:
    df = pd.read_excel('../../../data/kp_films/gendorf/Daily Hourly Process data.xlsx', sheet_name=sheet, header=4)    
    df['Line'] = sheet
    master = pd.concat([master, df], sort=False)
master = master[[i for i in master.columns if 'Unnamed' not in i]]
df = master

In [3]:
df = df[[i for i in df.columns if 'Unnamed' not in i]]

In [4]:
df['Run Time'] = df['Enddate'] - df['Startdate']
df['Yield'] = df['kg Net good'] / df['kg produced']
df['Rate'] = df['kg produced'] / (df['Run Time'].dt.total_seconds()/60/60)
df['different start end days'] = df['Startdate'].dt.day != df['Enddate'].dt.day
df['Uptime'] =  df['Startdate'] - df['Enddate'].shift() 
df['Uptime'] = df['Uptime'].dt.total_seconds()/60/60 # change to float (hrs)
df = df.loc[~(df['Uptime'] < 0)] #some times are wrong, have to select what makes sense
df = df.loc[df['Rate'] < 5e3] #some rates too high
# rename according to dashboard
df.columns = ['FA', 'Material', 'From Date/Time', 'To Date/Time', 'Net Quantity Produced', 'Quantity Good',
       'Product group', 'Polymer', 'Line', 'Run Time', 'Yield', 'Rate',
       'different start end days', 'Uptime']
oee = df # for asset_opportunity usage

load it

In [3]:
stats = pd.read_csv('../../../dashboards/product_margin/data/scores.csv', index_col=0)

In [4]:
stats = stats.loc[~(stats['descriptor'] == 'From Date/Time')].reset_index(drop=True)

In [5]:
stats.to_csv('../../../dashboards/product_margin/data/scores.csv')

In [6]:
oee = pd.read_csv('data/oee.csv')
oee['From Date/Time'] = pd.to_datetime(oee["From Date/Time"])
oee['To Date/Time'] = pd.to_datetime(oee["To Date/Time"])
oee["Run Time"] = pd.to_timedelta(oee["Run Time"])

In [7]:
df = pd.read_csv('../../../dashboards/product_margin/data/products.csv')
descriptors = df.columns[:8]
stat_df = pd.read_csv('../../../dashboards/product_margin/data/category_stats.csv')
old_products = df[descriptors].sum(axis=1).unique().shape[0]

In [8]:
weight_match = pd.read_csv('data/weight_match.csv')

figure out data link

In [9]:
list_of_stuff = []
for i in oee['Material']:
    list_of_stuff.append(i.split("-"))

In [10]:
dff = pd.DataFrame(list_of_stuff)

In [11]:
dff.head()

,0,1,2,3,4,5,6
0,LF,TG10F22,11G0000,100BJ,T52,50,4300R
1,LF,TG10F22,11G0000,100BJ,T52,45,4300R
2,LF,TG10F22,11G0000,100BJ,T52,45,690R
3,LF,TG10F22,11G0000,100BJ,T52,45,675R
4,LF,TG10F22,11G0000,100BJ,T52,40,711R


In [16]:
# dff.columns = [0,'letter + Base Type',2,3,'Additional Treatment','Thickness Material A',\
#               'Width Material Attri + F/R']

In [7]:
widths = []
for width in dff[6]:
    if any(s in width for s in df['Width Material Attri'].unique()):
        matching = [s for s in df['Width Material Attri'].unique() if s in width]
        widths.append(matching[0])
    else:
        widths.append(width)
        print(width)

 993F


In [8]:
thicknesses = []
for thickness in dff[5]:
    if any(s in thickness for s in df['Thickness Material A'].unique()):
        matching = [s for s in df['Thickness Material A'].unique() if s in thickness]
        thicknesses.append(matching[0])
    else:
        thicknesses.append(thickness)
        print(thickness)

 115
 195
 195


In [9]:
bases = []
for base in dff[1]:
    if any(s in base for s in df['Base Type'].unique()):
        matching = [s for s in df['Base Type'].unique() if s in base]
        bases.append(matching[0])
    else:
        bases.append(base)
        print(base)

TGSGV15
TG18V01
T888V03
CCCCV91
D253/58
D253/58
D253/58
D253/58
D253/58
D253/58


In [10]:
treatments = []
for treatment in dff[4]:
    if any(s in treatment for s in df['Additional Treatment'].unique()):
        matching = [s for s in df['Additional Treatment'].unique() if s in treatment]
        treatments.append(matching[0])
    else:
        treatments.append(treatment)
        print(treatment)

T10 


In [11]:
oee['Thickness Material A'] = thicknesses
oee['Width Material Attri'] = widths
oee['Base Type'] = bases
oee['Additional Treatment'] = treatments

In [12]:
oee['Product'] = oee['Polymer'] + '-' + bases + '-' + treatments

In [13]:
oee = oee.loc[oee['Rate'] < 2500]

In [46]:
# oee.to_csv('data/oee.csv', index=False)

In [47]:
# oee.to_csv('../../../dashboards/line_consolidation/data/oee.csv', index=False)

In [14]:
oee.loc[(oee['Product group'] == 'Dec. Surfaces/others') &
        (oee['Polymer'] == 'PVC') &
        (oee['Base Type'] == '225/05') &
        (oee['Additional Treatment'] == 'ZZZ')]['Line'].unique()

array(['K10', 'K18', 'K40'], dtype=object)

In [15]:
def make_product_sunburst(lines=['E27', 'E26']):
    fig = px.sunburst(oee.loc[oee['Line'].isin(lines)], path=['Product group', 'Polymer', 'Base Type', 'Additional Treatment', 'Line'], color='Line')
    return fig

In [16]:
lines=['K06', 'K40']
make_product_sunburst(lines)

In [17]:
def calculate_overlap(lines=['E27', 'E26']):
    path=['Product group', 'Polymer', 'Base Type', 'Additional Treatment']

    line1 = oee.loc[oee['Line'].isin([lines[0]])].groupby(path)['Quantity Good'].sum()
    line2 = oee.loc[oee['Line'].isin([lines[1]])].groupby(path)['Quantity Good'].sum()

    set1 = set(line1.index)
    set2 = set(line2.index)

    both = set1.intersection(set2)
    unique = set1.union(set2) - both

    kg_overlap = (line1.loc[list(both)].sum() + line2.loc[list(both)].sum()) /\
    (line1.sum() + line2.sum())
    return kg_overlap

In [18]:
calculate_overlap()

0.8235461890187693

In [19]:
plot = oee.loc[oee['Line'] == 'E26']
plot = plot.sort_values('Thickness Material A')
plot['Thickness Material A'] = pd.to_numeric(plot['Thickness Material A'])
px.density_contour(plot, x='Rate', y='Yield', 
             color='Thickness Material A')

In [20]:
desc = list(descriptors)

In [21]:
desc.append('EBIT')

In [22]:
descriptors[0]

'Product Family'

In [23]:
desc

['Product Family',
 'Product Group',
 'Base Polymer',
 'Base Type',
 'Additional Treatment',
 'Color Group',
 'Width Material Attri',
 'Thickness Material A',
 'EBIT']

In [24]:
px.parallel_categories(df[desc], color='EBIT')

In [25]:
oee.groupby(['Line', 'Thickness Material A'])['Rate'].describe()['std'].mean()

222.50132702881672

In [27]:
oee.groupby(['Line', 'Product'])['Rate'].describe()['std'].mean()

223.70497371374506

In [28]:
df['Product'] = df['Base Polymer'] + '-' + df['Base Type'] + '-' + df['Additional Treatment'] + '-' + df['Color Group']

In [34]:
oee.columns = ['FA', 'Material', 'From Date/Time', 'To Date/Time',
       'Net Quantity Produced', 'Quantity Good', 'Product Family', 'Base Polymer',
       'Line', 'Run Time', 'Yield', 'Rate', 'different start end days',
       'Uptime', 'Thickness Material A', 'Width Material Attri', 'Base Type',
       'Additional Treatment', 'Product']

In [38]:
@interact
def rate_dist_plot(line=oee.Line.unique(), pareto=['Product', 'Product Family', 'Thickness Material A']):
    fig, ax = plt.subplots(1,1,figsize=(10,10))
    data = oee.loc[oee['Line'] == line]
    for product in data[pareto].unique():
        if data.loc[data[pareto] == product].shape[0] > 3:
            data.loc[data[pareto] == product].plot.kde(by=pareto, y='Rate', ax=ax, label='{}, {}'.format(line,product))

interactive(children=(Dropdown(description='line', options=('E26', 'E27', 'E28', 'K06', 'K10', 'K17', 'K18', '…

In [37]:
@interact
def rate_dist_plot(line=oee.Line.unique(), pareto=['Product', 'Thickness Material A']):
    plot = oee.loc[oee['Line'] == line]
    plot = plot.sort_values('Thickness Material A')
    plot['Thickness Material A'] = pd.to_numeric(plot['Thickness Material A'])
    fig = px.density_contour(plot, x='Rate', y='Yield', 
                 color=pareto, marginal_x='rug', marginal_y='histogram')
    return fig

interactive(children=(Dropdown(description='line', options=('E26', 'E27', 'E28', 'K06', 'K10', 'K17', 'K18', '…

In [39]:
@interact
def rate_dist_plot(line=oee.Line.unique(), pareto=['Product', 'Thickness Material A'], metric=['Rate', 'Yield']):
    fig = px.histogram(oee.loc[oee['Line'] == line], x=metric, y=metric, 
                 color=pareto)
    return fig

interactive(children=(Dropdown(description='line', options=('E26', 'E27', 'E28', 'K06', 'K10', 'K17', 'K18', '…

Performance

In [40]:
downdays = pd.DataFrame(oee.groupby('Line')['Uptime'].sum().sort_values()/24)
downdays.columns = ['Unutilized Days, 2019']
px.bar(downdays, y=downdays.index, x='Unutilized Days, 2019', 
       orientation='h', color=downdays.index)

In [41]:
oee.groupby('Line')[[i for i in oee.columns if 'Quantity' in i]].sum()#.to_excel('line_kg_produced.xlsx')

,Net Quantity Produced,Quantity Good
Line,,
E26,3447725.631,2918350.412
E27,9505276.071,9438845.989
E28,1907804.857,1602759.425
K06,6512949.077,6203887.489
K10,6374532.791,5832571.703
K17,9478328.700,9023290.700
K18,7034534.508,6572817.608
K40,6795374.700,6248691.800


In [42]:
def calculate_margin_opportunity(sort='Worst', select=[0,10], descriptors=None):
    if sort == 'Best':
        local_df = stat_df.sort_values('score', ascending=False)
        local_df = local_df.reset_index(drop=True)
    else:
        local_df = stat_df
    if any(descriptors != None):
        local_df = local_df.loc[local_df['descriptor'].isin(descriptors)]
    if sort == 'Best':

        new_df = pd.DataFrame()
        for index in range(select[0],select[1]):
            x = df.loc[(df[local_df.iloc[index]['descriptor']] == \
                local_df.iloc[index]['group'])]
            new_df = pd.concat([new_df, x])
    else:

        new_df = df
        for index in range(select[0],select[1]):
            new_df = new_df.loc[~(new_df[local_df.iloc[index]['descriptor']] ==\
                    local_df.iloc[index]['group'])]

    new_EBIT = 1 / (new_df['Sales Quantity in KG'].sum() /
        df['Sales Quantity in KG'].sum()) * new_df['EBIT'].sum()
    
    EBIT_percent = (new_df['EBIT'].sum()) / df['EBIT'].sum() * 100
    new_products = new_df[descriptors].sum(axis=1).unique().shape[0]
    product_percent_reduction = (new_products) / \
        old_products * 100
    new_kg = new_df['Sales Quantity in KG'].sum()
    old_kg = df['Sales Quantity in KG'].sum()
    kg_percent = new_kg / old_kg * 100

    return "${:.1f} M of ${:.1f} M ({:.1f}%)".format(new_df['EBIT'].sum()/1e6, df['EBIT'].sum()/1e6, EBIT_percent), \
            "{} of {} Products ({:.1f}%)".format(new_products,old_products,product_percent_reduction),\
            "{:.1f} M of {:.1f} M kg ({:.1f}%)".format(new_kg/1e6, old_kg/1e6, kg_percent)

In [43]:
calculate_margin_opportunity(descriptors=descriptors)

('$9.6 M of $5.7 M (168.2%)',
 '2045 of 2643 Products (77.4%)',
 '42.6 M of 50.5 M kg (84.2%)')

In [58]:
oee.columns = ['FA', 'Material', 'From Date/Time', 'To Date/Time',
       'Net Quantity Produced', 'Quantity Good', 'Product Family', 'Base Polymer',
       'Line', 'Run Time', 'Yield', 'Rate', 'different start end days',
       'Uptime', 'Thickness Material A', 'Width Material Attri', 'Base Type',
       'Additional Treatment', 'Product']

In [44]:
df.loc[df['Base Polymer'] == 'PETG', 'Base Polymer'] = 'PET'
df.loc[df['Base Polymer'] == 'APET', 'Base Polymer'] = 'PET'
df.loc[df['Base Polymer'] == 'PS', 'Base Polymer'] = 'PO'
df.loc[df['Base Polymer'] == 'PP', 'Base Polymer'] = 'PO'

In [45]:
set(df['Product Family'].unique()) - set(oee['Product Family'].unique())

{'Mapping Default', 'Secondary business'}

In [47]:
set(oee['Product Family'].unique()) - set(df['Product Family'].unique())

{'PH Mono Films'}

In [48]:
path=['Product Family', 'Base Polymer', 'Base Type', 'Additional Treatment', 'Width Material Attri', 'Thickness Material A']
df.groupby(path)['EBIT'].describe()

count  \
Product Family Base Polymer Base Type Additional Treatment Width Material Attri Thickness Material A          
Cards Core     PET          G01T01    ZZZ                  468                  240                     1.0   
               PVC          146/19    ZZZ                  1010                 300                     1.0   
                                                           1285                 310                     1.0   
                                                           1300                 310                     1.0   
                                                           310                  320                     1.0   
...                                                                                                     ...   
Tape           PVC          240/15    ZZZ                  1915                 32                      1.0   
                                                           1985                 32                      1.0   
                                                           2120                 31                      1.0   
                                                           610                  35                      1.0   
                            240/91    ZZZ                  1280                 38                      1.0   

                                                                                                           mean  \
Product Family Base Polymer Base Type Additional Treatment Width Material Attri Thickness Material A              
Cards Core     PET          G01T01    ZZZ                  468                  240                     7747.34   
               PVC          146/19    ZZZ                  1010                 300                      -72.78   
                                                           1285                 310                    16619.76   
                                                           1300                 310                     2953.44   
                                                           310                  320                     1601.09   
...                                                                                                         ...   
Tape           PVC          240/15    ZZZ                  1915                 32                    -39769.22   
                                                           1985                 32                     -8508.18   
                                                           2120                 31                   -348973.68   
                                                           610                  35                      -670.87   
                            240/91    ZZZ                  1280                 38                       107.55   

                                                                                                      std  \
Product Family Base Polymer Base Type Additional Treatment Width Material Attri Thickness Material A        
Cards Core     PET          G01T01    ZZZ                  468                  240                   NaN   
               PVC          146/19    ZZZ                  1010                 300                   NaN   
                                                           1285                 310                   NaN   
                                                           1300                 310                   NaN   
                                                           310                  320                   NaN   
...                                                                                                   ...   
Tape           PVC          240/15    ZZZ                  1915                 32                    NaN   
                                                           1985                 32                    NaN   
                                                           2120         

In [49]:
oee.groupby(path)['Quantity Good'].describe()

count  \
Product Family Base Polymer Base Type Additional Treatment Width Material Attri Thickness Material A          
Cards Core     PET          G01T01    ZZZ                  468                  240                     6.0   
               PVC          146/19    ZZZ                  1285                 310                     3.0   
                                                           1300                 310                     1.0   
                                                           310                  320                     1.0   
                                                           475                  380                     2.0   
...                                                                                                     ...   
Tape           PVC          240/15    ZZZ                  830                  32                     50.0   
                                                                                33                      5.0   
                                                                                41                      1.0   
                                                           915                  32                     20.0   
                            240/91    ZZZ                  280                  38                      8.0   

                                                                                                              mean  \
Product Family Base Polymer Base Type Additional Treatment Width Material Attri Thickness Material A                 
Cards Core     PET          G01T01    ZZZ                  468                  240                    4511.333333   
               PVC          146/19    ZZZ                  1285                 310                    4162.933333   
                                                           1300                 310                    2433.500000   
                                                           310                  320                    2353.000000   
                                                           475                  380                    2010.500000   
...                                                                                                            ...   
Tape           PVC          240/15    ZZZ                  830                  32                    20022.222000   
                                                                                33                    21251.880000   
                                                                                41                     4188.000000   
                                                           915                  32                    20886.935000   
                            240/91    ZZZ                  280                  38                     2431.375000   

                                                                                                               std  \
Product Family Base Polymer Base Type Additional Treatment Width Material Attri Thickness Material A                 
Cards Core     PET          G01T01    ZZZ                  468                  240                    2034.714296   
               PVC          146/19    ZZZ                  1285                 310                    1029.040433   
                                                           1300                 310                            NaN   
                                                           310                  320                            NaN   
                                                           475                  380                      14.849242   
...                                                                                                            ...   
Tape           PVC          240/15    ZZZ                  830                  32                      539.033515   
                                                             

In [50]:
path=['Product Family', 'Base Polymer', 'Base Type', 'Additional Treatment', 'Width Material Attri', 'Thickness Material A']
merged = pd.merge(oee, df, how='inner', on=path)
merged.groupby('Line')['Adjusted EBITDA'].sum()/1e6

Line
E26     4.113330
E27    12.266399
E28     9.802457
K06    -9.577370
K10    15.812948
K17    21.249863
K18    11.952177
K40     1.816557
Name: Adjusted EBITDA, dtype: float64

In [53]:
df['Adjusted EBITDA'].sum()/1e6

9.57176192

In [54]:
path=['Line', 'Product Family', 'Base Polymer', 'Base Type', 'Additional Treatment', 'Width Material Attri', 'Thickness Material A']
oee['Run Time Hours'] = oee['Run Time'].dt.total_seconds()/60/60
grouped = pd.DataFrame(oee.groupby(path)[['Quantity Good', 'Run Time Hours', 'Net Quantity Produced']].sum())
grouped = grouped.reset_index()
print(grouped.shape)
path=['Product Family', 'Base Polymer', 'Base Type', 'Additional Treatment', 'Width Material Attri', 'Thickness Material A']
merged = pd.merge(grouped, df, how='inner', on=path)
merged = merged.loc[merged['Net Sales Quantity in KG'] > 0]
merged['Weight Adjusted EBITDA'] = merged['Adjusted EBITDA'] / merged['Net Sales Quantity in KG'] * merged['Quantity Good']
print(merged.shape)
print((merged.groupby('Line')["Adjusted EBITDA"].sum()/1e6).sum())
(merged.groupby('Line')["Adjusted EBITDA"].sum()/1e6)

(1442, 10)
(1134, 49)
4.318863189999999


Line
E26    1.233714
E27    2.450217
E28    0.415241
K06   -1.727362
K10    0.396213
K17    0.450998
K18    0.758163
K40    0.341678
Name: Adjusted EBITDA, dtype: float64

In [55]:
px.scatter(merged, 'Quantity Good', 'Net Sales Quantity in KG')

In [124]:
weight_match = merged.loc[abs(merged['Quantity Good'] - merged['Net Sales Quantity in KG']) < 1e4]
print(weight_match.shape)
print((weight_match.groupby('Line')["Adjusted EBITDA"].sum()/1e6).sum())
(weight_match.groupby('Line')["Adjusted EBITDA"].sum()/1e6)

(834, 49)
1.9128579700000001


Line
E26    0.339323
E27    0.546549
E28   -0.116727
K06   -0.794986
K10    0.332645
K17    0.044372
K18    1.438396
K40    0.123287
Name: Adjusted EBITDA, dtype: float64

In [125]:
weight_match['Rate'] = weight_match['Net Quantity Produced'] / weight_match['Run Time Hours']
weight_match['Yield'] = weight_match['Quantity Good'] / weight_match['Net Quantity Produced']
weight_match['EBITDA per Hr'] = weight_match['Adjusted EBITDA'] / weight_match['Run Time Hours']

In [126]:
px.scatter(weight_match, 'Quantity Good', 'Net Sales Quantity in KG')

In [115]:
weight_match.columns

Index(['Line', 'Product Family', 'Base Polymer', 'Base Type',
       'Additional Treatment', 'Width Material Attri', 'Thickness Material A',
       'Quantity Good', 'Run Time Hours', 'Net Quantity Produced',
       'Product Group', 'Color Group', 'Sales Quantity in KG',
       'Net Sales Quantity in KG', 'Gross Sales', 'Freight Revenue Subtotal',
       'Total Gross Sales Revenue', 'Sales Discounts', 'Term (Cash) Discounts',
       'Claims (Returns)', 'Bonus (Rebates)', 'Net Sales I',
       'Sales Price net per KG', 'Commissions Expense primary',
       'Commissions Expense secondary', 'Direct Sales Costs', 'Net Sales II',
       'Material Costs', 'Subcontracting', 'Added Value',
       'Packing Material (pallets only)', 'Packaging Overhead Variable',
       'Energy & Supplies Variable', 'Contribution Margin I - C',
       'Personnel Direct', 'Personnel Indirect', 'Maintenance Fix',
       'Other Fixed Overhead', 'Corporate Allocated Costs Prod',
       'Contribution Margin II - C', '

In [128]:
temp = pd.DataFrame(weight_match.groupby('Line')[["Adjusted EBITDA", "Run Time Hours", "Weight Adjusted EBITDA"]]\
                              .sum()).reset_index()
temp['Adjusted EBITDA / HR'] = temp['Weight Adjusted EBITDA'] / temp['Run Time Hours']
temp =temp.sort_values(by='Adjusted EBITDA')

In [117]:
ebitda_by_line = pd.DataFrame(weight_match.groupby('Line')["EBITDA per Hr"]\
                              .mean()).reset_index().sort_values('EBITDA per Hr')
ebitda_by_line.columns = ['Line', 'Adjusted EBITDA / HR']

In [118]:
ebitda_by_line

,Line,Adjusted EBITDA / HR
3,K06,-710.806414
2,E28,-530.247304
7,K40,-32.969473
4,K10,167.643405
5,K17,440.152137
6,K18,489.595220
0,E26,605.144770
1,E27,1049.516174


In [130]:
temp.groupby('Line').describe()

Adjusted EBITDA                                                      \
               count        mean std         min         25%         50%   
Line                                                                       
E26              1.0   339323.05 NaN   339323.05   339323.05   339323.05   
E27              1.0   546548.87 NaN   546548.87   546548.87   546548.87   
E28              1.0  -116727.49 NaN  -116727.49  -116727.49  -116727.49   
K06              1.0  -794986.23 NaN  -794986.23  -794986.23  -794986.23   
K10              1.0   332644.84 NaN   332644.84   332644.84   332644.84   
K17              1.0    44371.51 NaN    44371.51    44371.51    44371.51   
K18              1.0  1438396.49 NaN  1438396.49  1438396.49  1438396.49   
K40              1.0   123286.93 NaN   123286.93   123286.93   123286.93   

                             Run Time Hours                                \
             75%         max          count         mean std          min   
Line                                                                        
E26    339323.05   339323.05            1.0  1037.461389 NaN  1037.461389   
E27    546548.87   546548.87            1.0  1607.012500 NaN  1607.012500   
E28   -116727.49  -116727.49            1.0   182.488333 NaN   182.488333   
K06   -794986.23  -794986.23            1.0  1654.594444 NaN  1654.594444   
K10    332644.84   332644.84            1.0  2777.496667 NaN  2777.496667   
K17     44371.51    44371.51            1.0   606.837500 NaN   606.837500   
K18   1438396.49  1438396.49            1.0  2868.679444 NaN  2868.679444   
K40    123286.93   123286.93            1.0  1049.674444 NaN  1049.674444   

                                                          \
              25%          50%          75%          max   
Line                                                       
E26   1037.461389  1037.461389  1037.461389  1037.461389   
E27   1607.012500  1607.012500  1607.012500  1607.012500   
E28    182.488333   182.488333   182.488333   182.488333   
K06   1654.594444  1654.594444  1654.594444  1654.594444   
K10   2777.496667  2777.496667  2777.496667  2777.496667   
K17    606.837500   606.837500   606.837500   606.837500   
K18   2868.679444  2868.679444  2868.679444  2868.679444   
K40   1049.674444  1049.674444  1049.674444  1049.674444   

     Weight Adjusted EBITDA                                                \
                      count          mean std           min           25%   
Line                                                                        
E26                     1.0  3.192795e+05 NaN  3.192795e+05  3.192795e+05   
E27                     1.0  4.353744e+05 NaN  4.353744e+05  4.353744e+05   
E28                     1.0 -1.016651e+05 NaN -1.016651e+05 -1.016651e+05   
K06                     1.0 -7.332619e+05 NaN -7.332619e+05 -7.332619e+05   
K10                     1.0  3.491553e+05 NaN  3.491553e+05  3.491553e+05   
K17                     1.0  4.549499e+04 NaN  4.549499e+04  4.549499e+04   
K18                     1.0  1.262618e+06 NaN  1.262618e+06  1.262618e+06   
K40                     1.0  1.314650e+05 NaN  1.314650e+05  1.314650e+05   

                                               Adjusted EBITDA / HR  \
               50%           75%           max                count   
Line                                                                  
E26   3.192795e+05  3.192795e+05  3.192795e+05                  1.0   
E27   4.353744e+05  4.353744e+05  4.353744e+05                  1.0   
E28  -1.016651e+05 -1.016651e+05 -1.016651e+05                  1.0   
K06  -7.332619e+05 -7.332619e+05 -7.332619e+05                  1.0   
K10   3.491553e+05  3.491553e+05  3.491553e+05                  1.0   
K17   4.549499e+04  4.549499e+04  4.549499e+04                  1.0   
K18   1.262618e+06  1.262618e+06  1.262618e+06                  1.0   
K40   1.314650e+05  1.314650e+05  1.314650e+05                  1.0   

                             

In [129]:
px.bar(temp, 'Line', 'Adjusted EBITDA')

In [91]:
px.bar(temp, 'Line', 'Adjusted EBITDA / HR')

In [88]:
px.bar(ebitda_by_line, 'Line', 'Adjusted EBITDA / HR')

In [66]:
weight_match.columns

Index(['Line', 'Product Family', 'Base Polymer', 'Base Type',
       'Additional Treatment', 'Width Material Attri', 'Thickness Material A',
       'Quantity Good', 'Run Time Hours', 'Net Quantity Produced',
       'Product Group', 'Color Group', 'Sales Quantity in KG',
       'Net Sales Quantity in KG', 'Gross Sales', 'Freight Revenue Subtotal',
       'Total Gross Sales Revenue', 'Sales Discounts', 'Term (Cash) Discounts',
       'Claims (Returns)', 'Bonus (Rebates)', 'Net Sales I',
       'Sales Price net per KG', 'Commissions Expense primary',
       'Commissions Expense secondary', 'Direct Sales Costs', 'Net Sales II',
       'Material Costs', 'Subcontracting', 'Added Value',
       'Packing Material (pallets only)', 'Packaging Overhead Variable',
       'Energy & Supplies Variable', 'Contribution Margin I - C',
       'Personnel Direct', 'Personnel Indirect', 'Maintenance Fix',
       'Other Fixed Overhead', 'Corporate Allocated Costs Prod',
       'Contribution Margin II - C', '

In [72]:
weight_match['Thickness Material A'] = pd.to_numeric(weight_match['Thickness Material A'], errors='coerce')
weight_match['Width Material Attri'] = pd.to_numeric(weight_match['Width Material Attri'], errors='coerce')

In [114]:
weight_match.to_csv('data/weight_match.csv', index=False)

In [81]:
weight_match = pd.read_csv('data/weight_match.csv')

In [102]:
df['Adjusted EBITDA'].describe()

count      2643.000000
mean       3621.551994
std       26887.828846
min     -313112.630000
25%        -791.815000
50%          43.720000
75%        2904.465000
max      366433.330000
Name: Adjusted EBITDA, dtype: float64

In [106]:
weight_match = weight_match.sort_values('EBITDA per Hr').reset_index(drop=True)

In [109]:
weight_match['EBITDA per Hr Rank'] = weight_match.index

In [120]:
@interact
def bubble(y=['EBITDA per Hr', 'Adjusted EBITDA', 'Net Sales Quantity in KG'], 
           x=['Rate', 'Yield', 'EBITDA per Hr Rank', 'Adjusted EBITDA', 'Net Sales Quantity in KG'], 
           color=['Line', 'Thickness Material A', 'Width Material Attri', 'Product Family'],
           size=['Quantity Good', 'Net Sales Quantity in KG']):
    fig = px.scatter(weight_match, x=x, y=y, color=color, size=size)
    return fig

interactive(children=(Dropdown(description='y', options=('EBITDA per Hr', 'Adjusted EBITDA', 'Net Sales Quanti…

In [62]:
px.violin(weight_match, 'Line', 'Adjusted EBITDA', color='Line')

In [63]:
px.scatter(weight_match, 'Rate', 'Adjusted EBITDA', color='Product')

In [64]:
px.scatter(weight_match, 'Yield', 'Adjusted EBITDA', color='Product')

In [65]:
px.histogram(weight_match, 'Adjusted EBITDA', 'Adjusted EBITDA', color='Line')

In [4]:
df['Thickness Material A'] = pd.to_numeric(df['Thickness Material A'], errors='coerce')

In [11]:
df.columns

Index(['Product Family', 'Product Group', 'Base Polymer', 'Base Type',
       'Additional Treatment', 'Color Group', 'Width Material Attri',
       'Thickness Material A', 'Sales Quantity in KG',
       'Net Sales Quantity in KG', 'Gross Sales', 'Freight Revenue Subtotal',
       'Total Gross Sales Revenue', 'Sales Discounts', 'Term (Cash) Discounts',
       'Claims (Returns)', 'Bonus (Rebates)', 'Net Sales I',
       'Sales Price net per KG', 'Commissions Expense primary',
       'Commissions Expense secondary', 'Direct Sales Costs', 'Net Sales II',
       'Material Costs', 'Subcontracting', 'Added Value',
       'Packing Material (pallets only)', 'Packaging Overhead Variable',
       'Energy & Supplies Variable', 'Contribution Margin I - C',
       'Personnel Direct', 'Personnel Indirect', 'Maintenance Fix',
       'Other Fixed Overhead', 'Corporate Allocated Costs Prod',
       'Contribution Margin II - C', 'Research & Development',
       'Selling exp. (Condition)', 'General, & Adm

In [12]:
path=['Product Family', 'Base Polymer', 'Base Type', 'Additional Treatment', 'Width Material Attri', 'Thickness Material A']
merged = pd.merge(oee, df, how='inner', on=path)
merged.groupby('Line')['Adjusted EBITDA'].sum()/1e6

Line
E26     4.113330
E27    12.266399
E28     9.802457
K06    -9.577370
K10    15.812948
K17    21.249863
K18    11.952177
K40     1.816557
Name: Adjusted EBITDA, dtype: float64

In [17]:
(merged.groupby('Line')['Adjusted EBITDA'].sum()/1e6).sum()

67.43636092000014

In [14]:
merged['Net Sales est.'] = merged['Quantity Good'] * merged['Sales Price net per KG']

In [15]:
merged.groupby('Line')['Net Sales est.'].sum()/1e6

Line
E26     8.323012
E27    24.352448
E28     5.482169
K06    11.713358
K10    17.590493
K17     9.271638
K18    14.556552
K40     5.222391
Name: Net Sales est., dtype: float64

In [16]:
(merged.groupby('Line')['Net Sales est.'].sum()/1e6).sum()

96.5120619190416

In [21]:
merged = merged.loc[merged['Net Sales Quantity in KG'] > 0]

In [22]:
merged['EBIT per KG'] = merged['Adjusted EBITDA'] / merged['Net Sales Quantity in KG']

In [23]:
merged['Net Sales est.'] = merged['Quantity Good'] * merged['EBIT per KG']

In [24]:
(merged.groupby('Line')['Net Sales est.'].sum()/1e6)

Line
E26   -1.035312
E27    1.885680
E28    0.358856
K06   -1.626908
K10   -2.004173
K17    1.560612
K18   -0.499292
K40    0.424528
Name: Net Sales est., dtype: float64

In [29]:
[i for i in df.columns if 'Contribution' in i]

['Contribution Margin I - C', 'Contribution Margin II - C']

In [34]:
oee.groupby('Line')['Net Quantity Produced'].sum()

Line
E26    3447725.631
E27    9505276.071
E28    1907804.857
K06    6512949.077
K10    6374532.791
K17    9478328.700
K18    7034534.508
K40    6795374.700
Name: Net Quantity Produced, dtype: float64

In [35]:
oee.groupby('Line')['Yield'].describe()

,count,mean,std,min,25%,50%,75%,max
Line,,,,,,,,
E26,284.0,0.814500,0.208866,0.000000,0.726030,0.888447,0.972291,1.0
E27,718.0,0.992914,0.021956,0.775434,1.000000,1.000000,1.000000,1.0
E28,83.0,0.805310,0.194420,0.091749,0.743671,0.864632,0.942730,1.0
K06,679.0,0.950282,0.085190,0.000000,0.926668,1.000000,1.000000,1.0
K10,692.0,0.870968,0.181516,0.000000,0.828387,0.930709,1.000000,1.0
K17,471.0,0.954757,0.066043,0.599272,0.935730,0.981168,1.000000,1.0
K18,981.0,0.916422,0.131011,0.000000,0.884713,0.980515,1.000000,1.0
K40,1310.0,0.896432,0.127559,0.000000,0.852411,0.932461,1.000000,1.0


In [30]:
oee.groupby('Line')['Rate'].describe()

,count,mean,std,min,25%,50%,75%,max
Line,,,,,,,,
E26,284.0,926.123661,320.610333,3.789585,732.014929,981.659832,1127.601836,2041.318501
E27,719.0,1503.617463,322.186252,0.000000,1421.689631,1570.315464,1665.135966,2303.470926
E28,83.0,584.169512,248.075385,84.611653,456.122481,606.885117,735.254663,1533.942126
K06,679.0,1323.245275,277.587679,72.161147,1248.302877,1370.251734,1473.808251,2232.392679
K10,693.0,1027.941407,285.340577,0.000000,845.443880,999.927276,1196.884273,2383.379501
K17,471.0,1333.493533,295.124522,15.621593,1215.463977,1418.570773,1520.905615,1944.000000
K18,982.0,1205.641019,474.541590,0.000000,847.902917,1045.100882,1597.446650,2497.973708
K40,1311.0,1017.659426,208.817797,0.000000,891.925149,1015.681626,1150.387039,2094.944425


In [36]:
oee.columns

Index(['FA', 'Material', 'From Date/Time', 'To Date/Time',
       'Net Quantity Produced', 'Quantity Good', 'Product Family',
       'Base Polymer', 'Line', 'Run Time', 'Yield', 'Rate',
       'different start end days', 'Uptime', 'Thickness Material A',
       'Width Material Attri', 'Base Type', 'Additional Treatment', 'Product'],
      dtype='object')

In [27]:
merged.head()

,FA,Material,From Date/Time,To Date/Time,Net Quantity Produced,Quantity Good,Product Family,Base Polymer,Line,Run Time,Yield,Rate,different start end days,Uptime,Thickness Material A,Width Material Attri,Base Type,Additional Treatment,Product,Product Group,Color Group,Sales Quantity in KG,Net Sales Quantity in KG,Gross Sales,Freight Revenue Subtotal,Total Gross Sales Revenue,Sales Discounts,Term (Cash) Discounts,Claims (Returns),Bonus (Rebates),Net Sales I,Sales Price net per KG,Commissions Expense primary,Commissions Expense secondary,Direct Sales Costs,Net Sales II,Material Costs,Subcontracting,Added Value,Packing Material (pallets only),Packaging Overhead Variable,Energy & Supplies Variable,Contribution Margin I - C,Personnel Direct,Personnel Indirect,Maintenance Fix,Other Fixed Overhead,Corporate Allocated Costs Prod,Contribution Margin II - C,Research & Development,Selling exp. (Condition),"General, & Administrative (Condition)",EBITDA before Allocations,Adjusted EBITDA,Total Depreciation,EBIT,Net Sales est.,EBIT per KG
0,100779744,LF-TG10F22-11G0000-100BJ-T52 - 50-4300R,2018-12-30 21:35:36,2019-01-02 02:22:00,11762.078,11206.578,Shrink Sleeve,PET,E26,2 days 04:46:24,0.952772,222.879194,True,NaN,50,300,G10F22,T52,PET-G10F22-T52,Shrink Sleeve,GC,11790.0,11790.0,48328.80,0.0,48574.72,0.0,-614.96,0.00,-716.02,47243.74,4.007103,-200.13,0.0,-2908.31,44335.43,24665.04,0.0,22578.70,578.05,15.69,1087.16,17989.49,2871.12,2530.47,192.56,536.32,300.54,11558.48,588.32,1538.00,741.46,8690.70,8690.70,1790.37,6900.33,8260.645244,0.737125
1,100782578,LF-TG10F22-11G0000-100BJ-T52 - 50-4300R,2019-01-09 18:30:01,2019-01-11 09:51:42,24412.779,22611.000,Shrink Sleeve,PET,E26,1 days 15:21:41,0.926195,620.221483,True,0.0,50,300,G10F22,T52,PET-G10F22-T52,Shrink Sleeve,GC,11790.0,11790.0,48328.80,0.0,48574.72,0.0,-614.96,0.00,-716.02,47243.74,4.007103,-200.13,0.0,-2908.31,44335.43,24665.04,0.0,22578.70,578.05,15.69,1087.16,17989.49,2871.12,2530.47,192.56,536.32,300.54,11558.48,588.32,1538.00,741.46,8690.70,8690.70,1790.37,6900.33,16667.126183,0.737125
2,100814798,LF-TG10F22-11G0000-100BJ-T52 - 50-4300R,2019-03-30 03:03:05,2019-03-30 12:28:41,10049.905,9702.000,Shrink Sleeve,PET,E26,0 days 09:25:36,0.965382,1066.114392,False,0.0,50,300,G10F22,T52,PET-G10F22-T52,Shrink Sleeve,GC,11790.0,11790.0,48328.80,0.0,48574.72,0.0,-614.96,0.00,-716.02,47243.74,4.007103,-200.13,0.0,-2908.31,44335.43,24665.04,0.0,22578.70,578.05,15.69,1087.16,17989.49,2871.12,2530.47,192.56,536.32,300.54,11558.48,588.32,1538.00,741.46,8690.70,8690.70,1790.37,6900.33,7151.583664,0.737125
3,100814803,LF-TG10F22-11G0000-100BJ-T52 - 50-4300R,2019-04-21 13:38:16,2019-04-21 21:32:25,6470.135,5968.823,Shrink Sleeve,PET,E26,0 days 07:54:09,0.922519,818.745334,False,0.0,50,300,G10F22,T52,PET-G10F22-T52,Shrink Sleeve,GC,11790.0,11790.0,48328.80,0.0,48574.72,0.0,-614.96,0.00,-716.02,47243.74,4.007103,-200.13,0.0,-2908.31,44335.43,24665.04,0.0,22578.70,578.05,15.69,1087.16,17989.49,2871.12,2530.47,192.56,536.32,300.54,11558.48,588.32,1538.00,741.46,8690.70,8690.70,1790.37,6900.33,4399.766755,0.737125
4,100778032,LF-TG10F22-11G0000-100BJ-T52 - 45-4300R,2019-01-02 02:22:00,2019-01-03 02:46:14,30060.155,29419.000,Shrink Sleeve,PET,E26,1 days 00:24:14,0.978671,1231.777244,True,0.0,45,300,G10F22,T52,PET-G10F22-T52,Shrink Sleeve,GC,36847.7,36743.0,162107.39,0.0,163744.73,0.0,0.00,-438.69,-22.08,163283.96,4.443947,-3087.44,0.0,-8021.77,155262.19,76344.08,0.0,86939.88,3911.64,193.00,3494.27,71319.20,10617.18,6804.55,759.72,2071.86,947.88,50118.01,1838.70,4806.82,2317.33,41155.16,41155.16,5802.20,35352.96,32951.682009,1.120082


In [25]:
(merged.groupby('Line')['Net Sales est.'].sum()/1e6).sum()

-0.9360107305280252

In [46]:
merged.loc[merged['EBIT'] < 0].head()

,FA,Material,From Date/Time,To Date/Time,Net Quantity Produced,Quantity Good,Product Family,Base Polymer,Line,Run Time,Yield,Rate,different start end days,Uptime,Thickness Material A,Width Material Attri,Base Type,Additional Treatment,Product,Product Group,Color Group,Sales Quantity in KG,Net Sales Quantity in KG,Gross Sales,Freight Revenue Subtotal,Total Gross Sales Revenue,Sales Discounts,Term (Cash) Discounts,Claims (Returns),Bonus (Rebates),Net Sales I,Sales Price net per KG,Commissions Expense primary,Commissions Expense secondary,Direct Sales Costs,Net Sales II,Material Costs,Subcontracting,Added Value,Packing Material (pallets only),Packaging Overhead Variable,Energy & Supplies Variable,Contribution Margin I - C,Personnel Direct,Personnel Indirect,Maintenance Fix,Other Fixed Overhead,Corporate Allocated Costs Prod,Contribution Margin II - C,Research & Development,Selling exp. (Condition),"General, & Administrative (Condition)",EBITDA before Allocations,Adjusted EBITDA,Total Depreciation,EBIT,Net Sales est.
9,100780307,LF-TG10F22-11G0000-100BJ-T52 - 40- 711R,2019-01-03 08:39:16,2019-01-03 14:38:23,6481.000,5988.0,Shrink Sleeve,PET,E26,05:59:07,0.923931,1082.823595,False,0.000000,40,711,G10F22,T52,PET-G10F22-T52,Shrink Sleeve,TR,11160.1,11160.1,38075.02,0.0,39527.96,0.0,0.0,0.0,-1339.21,38188.75,3.42190,0.0,0.0,-1694.22,36494.53,23492.55,0.0,14696.20,2034.66,20.57,1696.17,9250.58,3132.63,2948.23,471.76,1025.10,460.70,1212.16,556.87,1455.83,701.86,-1502.40,-1502.40,2069.29,-3571.69,20490.339244
10,100881368,LF-TG10F22-11G0000-100BJ-T52 - 40- 711R,2019-09-23 06:51:29,2019-09-23 08:45:30,2900.000,2900.0,Shrink Sleeve,PET,E27,01:54:01,1.000000,1526.092677,False,0.000000,40,711,G10F22,T52,PET-G10F22-T52,Shrink Sleeve,TR,11160.1,11160.1,38075.02,0.0,39527.96,0.0,0.0,0.0,-1339.21,38188.75,3.42190,0.0,0.0,-1694.22,36494.53,23492.55,0.0,14696.20,2034.66,20.57,1696.17,9250.58,3132.63,2948.23,471.76,1025.10,460.70,1212.16,556.87,1455.83,701.86,-1502.40,-1502.40,2069.29,-3571.69,9923.510990
14,100782221,LF-TG10F12-11G0000-100_0-T45 - 40-1025R,2019-01-04 20:36:10,2019-01-05 03:50:44,4664.512,3557.2,Shrink Sleeve,PET,E26,07:14:34,0.762609,644.022521,True,0.000000,40,1025,G10F12,T45,PET-G10F12-T45,Shrink Sleeve,WH,511.9,511.9,2226.77,0.0,2226.77,0.0,0.0,0.0,0.00,2226.77,4.35001,0.0,0.0,-28.29,2198.48,1442.35,0.0,784.42,66.18,0.50,109.49,579.96,169.35,148.96,37.80,71.82,30.29,121.74,25.54,66.78,32.19,-2.77,-2.77,116.84,-119.61,15473.854745
16,100783526,LF-TG10F12-11G0000-100_0-T45 - 40-1025R,2019-01-13 06:38:36,2019-01-13 19:35:15,15028.700,14605.7,Shrink Sleeve,PET,E26,12:56:39,0.971854,1161.040366,False,0.000000,40,1025,G10F12,T45,PET-G10F12-T45,Shrink Sleeve,WH,511.9,511.9,2226.77,0.0,2226.77,0.0,0.0,0.0,0.00,2226.77,4.35001,0.0,0.0,-28.29,2198.48,1442.35,0.0,784.42,66.18,0.50,109.49,579.96,169.35,148.96,37.80,71.82,30.29,121.74,25.54,66.78,32.19,-2.77,-2.77,116.84,-119.61,63534.937662
18,100812457,LF-TG10F12-11G0000-100_0-T45 - 40-1025R,2019-03-31 15:38:44,2019-04-01 02:55:07,14069.920,13475.0,Shrink Sleeve,PET,E26,11:16:23,0.957717,1248.101717,True,2.566944,40,1025,G10F12,T45,PET-G10F12-T45,Shrink Sleeve,WH,511.9,511.9,2226.77,0.0,2226.77,0.0,0.0,0.0,0.00,2226.77,4.35001,0.0,0.0,-28.29,2198.48,1442.35,0.0,784.42,66.18,0.50,109.49,579.96,169.35,148.96,37.80,71.82,30.29,121.74,25.54,66.78,32.19,-2.77,-2.77,116.84,-119.61,58616.381618


In [23]:
path=['Product Family', 'Base Polymer', 'Base Type', 'Additional Treatment']#, 'Width Material Attri', 'Thickness Material A']
merged = pd.merge(oee, df, how='inner', on=path)
merged.groupby('Line')['EBIT'].sum()/1e6

Line
E26    137.080532
E27    621.537655
E28     -0.341353
K06   -442.030383
K10    -78.035379
K17    159.278833
K18    248.988446
K40    605.078995
Name: EBIT, dtype: float64

In [21]:
path=['Product Family', 'Base Polymer', 'Base Type', 'Additional Treatment', 'Width Material Attri']#, 'Thickness Material A']
merged = pd.merge(oee, df, how='outer', left_on=path, right_on=path)
merged.groupby('Line')['EBIT'].sum()/1e6

Line
E26     4.368711
E27    13.337225
E28     7.606417
K06   -14.472546
K10    11.172870
K17    20.274636
K18    27.344121
K40     2.892237
Name: EBIT, dtype: float64

In [16]:
path=['Product Family', 'Base Polymer', 'Base Type']#, 'Additional Treatment', 'Width Material Attri', 'Thickness Material A']
merged = pd.merge(oee, df, how='outer', left_on=path, right_on=path)
merged.groupby('Line')['EBIT'].sum()/1e6

Line
E26     185.876209
E27    1096.655998
E28      -0.341353
K06    -441.952139
K10      -7.831125
K17     159.278833
K18     245.622376
K40     737.750037
Name: EBIT, dtype: float64

In [15]:
path=['Product Family', 'Base Polymer']#, 'Base Type', 'Additional Treatment', 'Width Material Attri', 'Thickness Material A']
merged = pd.merge(oee, df, how='outer', on=path)
merged.groupby('Line')['EBIT'].sum()/1e6

Line
E26     710.022154
E27    2097.838537
E28      20.996202
K06   -1876.130689
K10     -66.927846
K17     435.388915
K18     910.532316
K40    3346.596207
Name: EBIT, dtype: float64

In [14]:
path=['Product Family', 'Base Polymer']#, 'Base Type', 'Additional Treatment', 'Width Material Attri', 'Thickness Material A']
merged = pd.merge(oee, df, how='outer', left_on=path, right_on=path)
merged.groupby('Line')['EBIT'].sum()/1e6

Line
E26     710.022154
E27    2097.838537
E28      20.996202
K06   -1876.130689
K10     -66.927846
K17     435.388915
K18     910.532316
K40    3346.596207
Name: EBIT, dtype: float64

In [423]:
oee.loc[oee['Line'].isin(['E28', 'E27', 'E26'])]['Base Polymer'].unique()

array(['PET', 'PO'], dtype=object)

In [424]:
df['Base Polymer'].unique()

array(['PETG', 'PVC', 'APET', 'PP', '#', 'PS'], dtype=object)

In [418]:
pd.merge?

In [410]:
merged['Sales Price'] = merged['Sales Price net per KG'] * merged['Quantity Good']
merged['EBIT per batch'] = merged['EBIT'] / merged['Net Sales Quantity in KG'] * merged['Quantity Good']